### Preparando o ambiente

In [1]:
%matplotlib inline

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets


In [2]:
import tensorflow.compat.v1 as tf_1
tf_1.compat.v1.disable_eager_execution()

In [3]:
mnist = tensorflow_datasets.load('mnist')


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


### Modelo

#### Configurações

O codificador mapeia uma imagem para uma distribuição
     sobre códigos plausíveis para essa imagem. 
     E reflete o que o código deve ser para dada uma imagem.

In [4]:
def make_encoder(data, tamanho):
    x = tf.layers.flatten(data)
    x = tf.layers.dense(x, 200, tf.nn.relu)
    x = tf.layers.dense(x, 200, tf.nn.relu)
    loc = tf.layers.dense(x, tamanho)
    scale = tf.layers.dense(x, tamanho, tf.nn.softplus)
    return tfd.MultivariateNormalDiag(loc, scale)

Definição de qual distribuição de
     códigos que esperamos.  
     Isso fornece uma restrição 
     sobre quais códigos o VAE pode usar. Muitas vezes é apenas uma
     distribuição Normal com média zero e variância unitária.

In [5]:
def make_prior(tamanho):
    loc = tf.zeros(tamanho)
    scale = tf.ones(tamanho)
    return tfd.MultivariateNormalDiag(loc, scale)

O decodificador pega um código e o mapeia de volta  uma
     distribuição de imagens que são plausíveis para o código.
     Permite reconstruir as imagens, ou gerar novas
      para qualquer código que escolhermos.
     Aqui, é utilizado a distribuição de Bernoulli para os dados,
     considerando uma modelagem de pixels como valores binários. Dependendo do tipo
     e domínio de seus dados, você pode querer modelá-los de uma
     maneira diferente

No TensorFlow ao utilizar a distribuição bernoulli informamos que as duas dimensões internas, largura e altura no nosso caso, pertencem ao mesmo ponto de dados, embora tenham parâmetros independentes. Isso nos permite avaliar a probabilidade de uma imagem sob a distribuição, não apenas pixels individuais.

In [6]:
def make_decoder(code, data_shape):
    x = tf.layers.dense(code, 200, tf.nn.relu)
    x = tf.layers.dense(x, 200, tf.nn.relu)
    logit = tf.layers.dense(x, np.prod(data_shape))
    logit = tf.reshape(logit, [-1] + data_shape)
    return tfd.Independent(tfp.Bernoulli(logit), 2)

Vizualização

In [7]:
def plot_codes(ax, codes, labels):
    ax.scatter(codes[:, 0], codes[:, 1], s=2, c=labels, alpha=0.1)
    ax.set_aspect('equal')
    ax.set_xlim(codes.min() - .1, codes.max() + .1)
    ax.set_ylim(codes.min() - .1, codes.max() + .1)
    ax.tick_params(
          axis='both', which='both', left='off', bottom='off',
          labelleft='off', labelbottom='off')

In [8]:
def plot_samples(ax, samples):
    for index, sample in enumerate(samples):
        ax[index].imshow(sample, cmap='gray')
        ax[index].axis('off')

##### Execução

In [9]:
make_encoder = tf_1.make_template('encoder', make_encoder)
make_decoder = tf_1.make_template('decoder', make_decoder)

In [10]:
data = tf_1.placeholder(tf.float32, [None, 28, 28])
prior = make_prior(tamanho=2)
posterior = make_encoder(data, tamanho=2)
code = posterior.sample()
likelihood = make_decoder(code, [28, 28]).log_prob(data)
divergence = tfd.kl_divergence(posterior, prior)
elbo = tf.reduce_mean(likelihood - divergence)

AttributeError: ignored